In [106]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import djvu.decode

def isArmenian(s):
    return any(unicodeblock.blocks.of(c) == 'ARMENIAN' for c in s)

def isCyrillic(s):
    return any(unicodeblock.blocks.of(c) == 'CYRILLIC' for c in s)

print(djvu.decode.DDJVU_VERSION)

26


In [107]:
context = djvu.decode.Context()
document = context.new_document(djvu.decode.FileUri('data/hayeren.djvu'))
document.decoding_job.wait()

In [204]:
pages = []

for page in document.pages:
    val = page.text.sexpr.value

    line_delims_and_words = [y for x in val if type(x) == tuple for y in x if type(y) != int]

    lines = []
    current_line = []

    for entry in line_delims_and_words[1:]:
        if type(entry) == tuple:
            current_line.append(entry[-1])
        else:
            lines.append(' '.join(current_line))
            current_line = []
    lines.append(' '.join(current_line))

    pages.append(lines)
    
core_lines = [line.strip() for page in pages[16:-1] for line in page[:-2]]

# some lines of the form 'остановить (для проверки) կանգնեցնել' mix Cyrillic and Armenian!
core_lines_fix = []
for line in core_lines:
    if isArmenian(line) and isCyrillic(line):
        core_lines_fix.append(line[:line.index(')')+1])
        core_lines_fix.append(line[line.index(')')+2:])
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

# another fix, for consecutive Armenian lines. use separate loop since logic is more complex
core_lines_fix = []
skip_steps = 0

for i in range(len(core_lines)):
    if skip_steps:
        skip_steps -= 1
        continue
    
    line = core_lines[i]
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isArmenian(line) and isArmenian(next_line):
        if next_line.startswith(line):
            pass
        elif isArmenian(core_lines[i+2]): # only one case!
            core_lines_fix.append(line + ' ' + next_line + ' ' + core_lines[i+2])
            skip_steps = 2
        else:
            core_lines_fix.append(line + ' ' + next_line)
            skip_steps = 1
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

# one more fix, for consecutive Russian lines (other than headers)
core_lines_fix = []
skip_steps = 0

for i in range(len(core_lines)):
    if skip_steps:
        skip_steps -= 1
        continue
    
    line = core_lines[i]
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isCyrillic(line) and isCyrillic(next_line):
        if line in section_headers:
            core_lines_fix.append(line)
        else:
            core_lines_fix.append(line + ' ' + next_line)
            skip_steps = 1
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

del core_lines_fix

# get section headers from pages 4-11, ignore page numbers in page headers
section_headers = [line for 
    page in pages[4:12] for line in page[1:-1] if any(not c in '0123456789' for c in line)][3:]

section_headers

['ОСНОВНЫЕ ПОНЯТИЯ',
 'основные понятия | часть 1',
 '1. Местоимения',
 '2. Приветствия. Прощания. Извинения. Благодарность',
 '3. Обращения',
 '4. Числаот1до100',
 '5. Числа от 100',
 '6. Числа. Порядковые числительные',
 '7. Числа. Дроби',
 '8. Числа. Математические действия',
 '9. Числа. Разное',
 '10. Самые важные глаголы - 1',
 '11. Самые важные глаголы - 2',
 '12. Самые важные глаголы - 3',
 '13. Самые важные глаголы - 4',
 '14. Цвета',
 '15. Вопросы',
 '16. Основные предлоги',
 '17. Вводные и служебные слова. Наречия - 1',
 '18. Вводные и служебные слова. Наречия - 2',
 'основные понятия | часть 2',
 '19. Дни недели',
 '20. Часы. Время суток',
 '21. Месяцы. Времена года',
 '22. Время. Разное',
 '23. Противоположности',
 '24. Линии и формы',
 '25. Меры измерения',
 '26. Ёмкости',
 '27. Материалы',
 '28. Металлы',
 'ЧЕЛОВЕК',
 'человек | тело человека',
 '29. Человек. Общие понятия',
 '30. Анатомия',
 '31. Голова',
 '32. Тело',
 'одежда | аксессуары',
 '33. Верхняя одежда',
 '34. 

In [193]:
# check that all section_headers do actually appear in text
for sh in section_headers:
    print('✅' if sh in core_lines else '❌', sh)

✅ ОСНОВНЫЕ ПОНЯТИЯ
✅ основные понятия | часть 1
✅ 1. Местоимения
✅ 2. Приветствия. Прощания. Извинения. Благодарность
✅ 3. Обращения
✅ 4. Числаот1до100
✅ 5. Числа от 100
✅ 6. Числа. Порядковые числительные
✅ 7. Числа. Дроби
✅ 8. Числа. Математические действия
✅ 9. Числа. Разное
✅ 10. Самые важные глаголы - 1
✅ 11. Самые важные глаголы - 2
✅ 12. Самые важные глаголы - 3
✅ 13. Самые важные глаголы - 4
✅ 14. Цвета
✅ 15. Вопросы
✅ 16. Основные предлоги
✅ 17. Вводные и служебные слова. Наречия - 1
✅ 18. Вводные и служебные слова. Наречия - 2
✅ основные понятия | часть 2
✅ 19. Дни недели
✅ 20. Часы. Время суток
✅ 21. Месяцы. Времена года
✅ 22. Время. Разное
✅ 23. Противоположности
✅ 24. Линии и формы
✅ 25. Меры измерения
✅ 26. Ёмкости
✅ 27. Материалы
✅ 28. Металлы
✅ ЧЕЛОВЕК
✅ человек | тело человека
✅ 29. Человек. Общие понятия
✅ 30. Анатомия
✅ 31. Голова
✅ 32. Тело
✅ одежда | аксессуары
✅ 33. Верхняя одежда
✅ 34. Одежда
✅ 35. Одежда. Бельё
✅ 36. Головной убор
✅ 37. Обувь
✅ 38. Ткани. Матери

In [226]:
category = ''
subcategory = ''
group = ''

dfRows = []

for i in range(len(core_lines)):
    line = core_lines[i]
    
    if isArmenian(line):
        continue # handle all processing one line ahead of definitions
    
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isCyrillic(line) and isCyrillic(next_line):
        if re.match(r'^\d+\. ', line):
            group = line
        if line.islower():
            subcategory = line
        if line.isupper():
            category = line
    
    if isArmenian(next_line):
        dfRows.append({
            'category': category, 'subcategory': subcategory, 'group': group,
            'russian': line, 'armenian': next_line
        })

DataFrame(dfRows)

,category,subcategory,group,russian,armenian
0,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,1. Местоимения,я,եես
1,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,1. Местоимения,ты,դու
2,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,1. Местоимения,"он, она, оно",նա
3,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,1. Местоимения,мы,մենք
4,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 1,1. Местоимения,вы,դուք
...,...,...,...,...,...
8808,РАЗНОЕ,основные глаголы | 550 слов,257. Глаголы У-Я,цитировать,մեջբերել
8809,РАЗНОЕ,основные глаголы | 550 слов,257. Глаголы У-Я,чистить,սրբել
8810,РАЗНОЕ,основные глаголы | 550 слов,257. Глаголы У-Я,читать,կարդալ
8811,РАЗНОЕ,основные глаголы | 550 слов,257. Глаголы У-Я,чувствовать (опасность),զգալ


In [222]:
core_lines[10910:10929]

['ծանծաղուտ',
 'сесть на мель',
 'ծանծաղուտ ընկնել',
 'буря',
 'փոթորիկ',
 'сигнал',
 'ազդանշան',
 'тонуть (о корабле)',
 'խորտակվել',
 'SOS',
 'SO՜S',
 'спасательный круг',
 'փրկագոտի',
 '172. Аэропорт',
 'аэропорт',
 'օդանավակայան',
 'самолёт',
 'ինքնաթիռ',
 'авиакомпания']

In [224]:
isArmenian('SO՜S')

True